<a href="https://colab.research.google.com/github/nadia-maarfavi/YouTube.Data.Scraper/blob/main/GetComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import json
from apiclient.discovery import build
import pandas as pd
import numpy as np

In [69]:
youTubeApiKey="...." #Get YouTube API Key using this instruction: https://developers.google.com/youtube/v3/getting-started#
youtube=build('youtube','v3',developerKey=youTubeApiKey)


In [70]:
def get_comments(videoId):
    comments=[]
    commentsId = []
    repliesCount= []
    likesCount=[]
    PublishDate = []
    global df
    df = pd.DataFrame(columns=('Comment', 'CommentID', 'Reply','Like','PulishDate'))

    try:
      # make an API call using service
      response = youtube.commentThreads().list(
          part="snippet",
          maxResults=100,
          order="time",
          textFormat="plainText",
          videoId=videoId
      ).execute()


      while response: # this loop will continue to run until you max out your quota
          for item in response['items']:
            # index item for desired data features
              comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
              comment_id = item['snippet']['topLevelComment']['id']
              reply_count = item['snippet']['totalReplyCount']
              like_count = item['snippet']['topLevelComment']['snippet']['likeCount']
              Date = item['snippet']['topLevelComment']['snippet']['publishedAt']

            # append to lists
              comments.append(comment)
              commentsId.append(comment_id)
              repliesCount.append(reply_count)
              likesCount.append(like_count)
              PublishDate.append(Date)

              try:
                df = df._append({'Comment': comment, 'CommentID': comment_id,'Reply':reply_count,'Like':like_count,'PulishDate':Date}, ignore_index=True)
              except Exception as error:
                print("An exception occurred:", error)

        # check for nextPageToken, and if it exists, set response equal to the JSON response
          if 'nextPageToken' in response:
              print("Found")
              response = youtube.commentThreads().list(
                  part='snippet',
                  maxResults=100,
                  textFormat="plainText",
                  order="time",
                  videoId=videoId,
                  pageToken=response['nextPageToken']
              ).execute()
          else:
              break

      # return data of interest
      return {
          'Comments': comments,
          'Comment ID': commentsId,
          'Reply Count' : repliesCount,
          'Like Count' : likesCount,
          'Publish Date':PublishDate}

    except:
       return {
          'Comments': np.nan,
          'Comment ID': np.nan,
          'Reply Count' : np.nan,
          'Like Count' : np.nan,
          'Publish Date':np.nan}

In [71]:
result = get_comments('Mw4dkqC3C3o')
df

,Comment,CommentID,Reply,Like,PulishDate
0,Ты украл видео ты украл ты обманщик,UgwqDBMGKSrMfbbw3iZ4AaABAg,0,0,2023-09-22T11:25:35Z
1,"😉 ""promo sm""",Ugx8ce9ogQAptPdWd-N4AaABAg,0,4,2023-09-21T15:02:39Z
2,Transforming a simple paper into a work of art...,Ugza2ca83tvxJYWJGU94AaABAg,0,2,2023-08-29T16:28:34Z
3,This is awesome ❤,UgxHi7wpGgOqRXk9iUx4AaABAg,0,2,2023-08-29T16:25:10Z
